In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
# device = 'cpu'

print(device)

cuda


# Собираем датасет

Let's have a look at the different NER tags. 

We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [3]:
pth = '/home/sondors/Documents/price/BERT_NER/csv/NER_2609.csv'
df_csv = pd.read_csv(pth, sep=';')
df_csv

,width,height,radius,brand,offer,type,line,BIO_Tags
0,175,65.0,14,Amtel,Amtel 175 / 65 R 14 82Q Nordmaster (В-228),Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"
1,175,70.0,13,Avatyre,Avatyre 175 / 70 R 13 82Q Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
2,205,55.0,16,Avatyre,Avatyre 205 / 55 R 16 91T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
3,215,65.0,16,Avatyre,Avatyre 215 / 65 R 16 98T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
4,215,65.0,16,Avatyre,Avatyre 215 / 65 R 16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O,O"
...,...,...,...,...,...,...,...,...
7013,"5,5",NaN,14,СКАД,"СКАД Эверест 5,5 \ R 14 4*100 ET38 d67,1 [0830...",диски,Эверест,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O"
7014,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0 \ R 17 5*100 ET46 d56,1 Алмаз [...",диски,Эссен,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O,O"
7015,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0 \ R 17 5*100 ET46 d56,1 [2850508]",диски,Эссен,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O"
7016,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0 \ R 17 5*108 ET50 d63,35 Алмаз ...",диски,Эссен,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O,O"


In [4]:
label2id = {'B-width': 1,
            'B-height': 2,
            'B-radius': 3,
            'B-brand': 4,
            'B-line': 5,
            'O': 0}

label2id

{'B-width': 1, 'B-height': 2, 'B-radius': 3, 'B-brand': 4, 'B-line': 5, 'O': 0}

In [5]:
id2label = dict((v,k) for k,v in label2id.items())
id2label

{1: 'B-width', 2: 'B-height', 3: 'B-radius', 4: 'B-brand', 5: 'B-line', 0: 'O'}

Let's verify that a random sentence and its corresponding tags are correct:

In [6]:
df_csv.iloc[41].offer

'Bridgestone 175 / 70 R 13 82T Ice Cruiser 7000'

In [7]:
df_csv.iloc[41].BIO_Tags

'B-brand,B-width,O,B-height,O,B-radius,O,O,O,O'

# Дальше идет обучение

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [8]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
# model_pth = 'bert-base-uncased'
model_pth = "/home/sondors/29887"
tokenizer = BertTokenizer.from_pretrained(model_pth)

A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level! 

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then we will have to propagate the word’s original label to all of its wordpieces: "b-gpe", "b-gpe", "b-gpe". The model should be able to produce the correct labels for each individual wordpiece. The function below (taken from [here](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118)) implements this.






In [9]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

Note that this is a **design decision**. You could also decide to only label the first wordpiece of each word and let the model only learn this (this is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063)). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word.

All of them lead to good performance.

Next, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. 

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).  

In [10]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.offer[index]  
        word_labels = self.data.BIO_Tags[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [11]:
train_size = 0.8
train_dataset = df_csv.sample(frac=train_size,random_state=200)
test_dataset = df_csv.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_csv.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (7018, 8)
TRAIN Dataset: (5614, 8)
TEST Dataset: (1404, 8)


Let's have a look at the first training example:

In [12]:
training_set[0]

{'ids': tensor([   2,    1, 7623,   19,  679,    1,  757,    1,    1,    1,    1,    3,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0, 

Let's verify that the input ids and corresponding targets are correct:

In [13]:
training_set[0]["ids"]

tensor([   2,    1, 7623,   19,  679,    1,  757,    1,    1,    1,    1,    3,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [14]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
[UNK]       B-brand
235         B-width
/           O
60          B-height
[UNK]       O
18          B-radius
[UNK]       O
[UNK]       O
[UNK]       O
[UNK]       O
[SEP]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


Now, let's define the corresponding PyTorch dataloaders:

In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [16]:
model = BertForTokenClassification.from_pretrained(model_pth, 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /home/sondors/29887 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwi

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83. 

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence (in other words, for all of the 512 tokens). The mean of 512 times -log(1/17) is, you guessed it, -log(1/17).  

Let's verify this:



In [17]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.8606, device='cuda:0', grad_fn=<NllLossBackward0>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [18]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 6])

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [19]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [20]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [21]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

Training epoch: 1
Training loss per 100 training steps: 1.7402805089950562
Training loss per 100 training steps: 0.6877573082352629
Training loss epoch: 0.45500595516271214
Training accuracy epoch: 0.6721613916666889
Training epoch: 2
Training loss per 100 training steps: 0.10553861409425735
Training loss per 100 training steps: 0.08213349121927034
Training loss epoch: 0.07468506530858576
Training accuracy epoch: 0.7158075691154229
Training epoch: 3
Training loss per 100 training steps: 0.060359179973602295
Training loss per 100 training steps: 0.05684349497798646
Training loss epoch: 0.05500286470421336
Training accuracy epoch: 0.7158746595112265
Training epoch: 4
Training loss per 100 training steps: 0.04913107678294182
Training loss per 100 training steps: 0.04719728725676489
Training loss epoch: 0.04520406495695087
Training accuracy epoch: 0.7158068467960688
Training epoch: 5
Training loss per 100 training steps: 0.0453915111720562
Training loss per 100 training steps: 0.0383883394

And let's train the model!

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.3213791847229004


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 for epoch in range(EPOCHS):                                                                  │
│   2 │   print(f"Training epoch: {epoch + 1}")                                                    │
│ ❱ 3 │   train(epoch)                                                                             │
│   4                                                                                              │
│                                                                                                  │
│ in train:15                                                                                      │
│                                                                                                  │
│   12 │   │   mask = batch['mask'].to(device, dtype = torch.long)                                 │
│   13 │   │   targets = batch['targets'].to(device, dtype = torch.long)                           │
│   14 │   │                                                                                       │
│ ❱ 15 │   │   outputs = model(input_ids=ids, attention_mask=mask, labels=targets)                 │
│   16 │   │   loss, tr_logits = outputs.loss, outputs.logits                                      │
│   17 │   │   tr_loss += loss.item()                                                              │
│   18                                                                                             │
│                                                                                                  │
│ /home/sondors/anaconda3/envs/torch_gpu/lib/python3.8/site-packages/torch/nn/modules/module.py:11 │
│ 94 in _call_impl                                                                                 │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/sondors/anaconda3/envs/torch_gpu/lib/python3.8/site-packages/transformers/models/bert/mode │
│ ling_bert.py:1771 in forward                                                                     │
│                                                                                                  │
│   1768 │   │   loss = None                                                                       │
│   1769 │   │   if labels is not None:                                                            │
│   1770 │   │   │   loss_fct = CrossEntropyLoss()                                                 │
│ ❱ 1771 │   │   │   loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))            │
│   1772 │   │                                                                                     │
│   1773 │   │   if not return_dict:                                                               │
│   1774 │   │   │   output = (logits,) + outputs[2:]                                              │
│                                                            

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits. 

In [22]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [23]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.027167487889528275
Validation Loss: 0.028900550022213298
Validation Accuracy: 0.7134909264450868


However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library: 

In [24]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

       brand       0.00      0.00      0.00      1526
      height       1.00      0.00      0.00      1238
        line       0.00      0.00      0.00       449
      radius       0.40      0.00      0.01      1453
       width       1.00      0.00      0.00      1725

   micro avg       0.57      0.00      0.00      6391
   macro avg       0.48      0.00      0.00      6391
weighted avg       0.55      0.00      0.00      6391



/home/sondors/anaconda3/envs/torch_gpu/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences. 
Here, we use the prediction of the **first word piece of every word**. Note that the function we used to train our model (`tokenze_and_preserve_labels`) propagated the label to all subsequent word pieces (so you could for example also perform a majority vote on the predicted labels of all word pieces of a word).

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

In [25]:
test_dataset

,width,height,radius,brand,offer,type,line,BIO_Tags
0,205,55.0,16,Avatyre,Avatyre 205 / 55 R 16 91T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
1,215,55.0,17,BFGoodrich,BFGoodrich 215 / 55 R 17 98Q G-Force Stud (XL),Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O"
2,215,60.0,16,BFGoodrich,BFGoodrich 215 / 60 R 16 99Q G-Force Stud (XL),Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O"
3,35,12.0,15,BFGoodrich,"BFGoodrich 35 / 12,5 R 15 113Q Mud Terrain T /...",Колобокс_Шины,NaN,"B-brand,B-width,O,O,O,B-radius,O,O,O,O,O,O,O"
4,185,60.0,14,Bridgestone,Bridgestone 185 / 60 R 14 82S Blizzak Revo-GZ,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"
...,...,...,...,...,...,...,...,...
1399,"6,5",NaN,16,СКАД,"СКАД Турин 6,5 \ R 16 5*108 ET50 d63,35 [1980808]",диски,Турин,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O"
1400,"6,5",NaN,16,СКАД,"СКАД Турин 6,5 \ R 16 5*114,3 ET40 d66,1 Алмаз...",диски,Турин,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O,O"
1401,"6,5",NaN,16,СКАД,"СКАД Турин 6,5 \ R 16 5*114,3 ET45 d60,1 [1980...",диски,Турин,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O"
1402,"7,0",NaN,17,СКАД,"СКАД Турин 7,0 \ R 17 5*100 ET48 d56,1 Алмаз [...",диски,Турин,"B-brand,B-line,B-width,O,O,B-radius,O,O,O,O,O"


In [29]:
train_dataset

,width,height,radius,brand,offer,type,line,BIO_Tags
0,235,60.0,18,Sailun,Sailun 235 / 60 R 18 103H Ice Blazer WST1,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O"
1,245,70.0,17,Maxxis,Maxxis 245 / 70 R 17 110S AT771 Bravo,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"
2,185,65.0,14,Contyre,Contyre 185 / 65 R 14 86T Arctic Ice II,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O"
3,215,45.0,17,Bridgestone,Bridgestone 215 / 45 R 17 87S Blizzak VRX,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"
4,215,65.0,16,НК.ШЗ,Viatti Bosco S / T V-526 215 / 65 / 16 T 98,легковая_резина,Bosco S/T V-526,"O,O,O,O,O,O,B-width,O,B-height,O,B-radius,O,O"
...,...,...,...,...,...,...,...,...
5609,265,65.0,17,Michelin,Michelin 265 / 65 R 17 116T Latitude X-Ice Nor...,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O,O,O"
5610,195,75.0,16,Нокиан,Нокиан 195 / 75 / 16 S 107 / 105 C HAKKA C 2,легковая_резина,HAKKA C 2,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O,..."
5611,225,60.0,18,Bridgestone,Bridgestone 225 / 60 R 18 100H Dueler H / L 33,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O,O,O"
5612,215,50.0,17,Nokian,Nokian 215 / 50 R 17 95T Hakkapeliitta 7,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"


In [35]:
sentence = "BFGoodrich 215 / 55 R 17 98Q G-Force Stud (XL)"
sentence = "СКАД Турин 6,5 \ R 16 5*114,3 ET40 d66,1 Алмаз"
sentence = "Avatyre 205 / 55 R 16 91T Freeze"

sentence = "Viatti Bosco S / T V-526 215 / 65 / 16 T 98"

def token_classification(sentence):
    inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    word_level_predictions = []
    for pair in wp_preds:
        if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
            # skip prediction
            continue
        else:
            word_level_predictions.append(pair[1])

    # we join tokens, if they are not special ones
    str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
    print(str_rep)
    print(word_level_predictions)

i = 0
for sentence in list(train_dataset['offer']):
    print(sentence)
    token_classification(sentence)
    i += 1
    if i == 10000:
        break

Sailun 235 / 60 R 18 103H Ice Blazer WST1
[UNK] 235 / 60 [UNK] 18 [UNK] [UNK] [UNK] [UNK]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Maxxis 245 / 70 R 17 110S AT771 Bravo
[UNK] 245 / 70 [UNK] 17 [UNK] [UNK] [UNK]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Contyre 185 / 65 R 14 86T Arctic Ice II
[UNK] 185 / 65 [UNK] 14 [UNK] [UNK] [UNK] [UNK]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Bridgestone 215 / 45 R 17 87S Blizzak VRX
[UNK] 215 / 45 [UNK] 17 [UNK] [UNK] [UNK]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Viatti Bosco S / T V-526 215 / 65 / 16 T 98
[UNK] [UNK] [UNK] / [UNK] [UNK] - 526 215 / 65 / 16 [UNK] 98
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Laufenn 205 / 75 R 16C 110 / 108 R iFIT VAN (LY31)
[UNK] 205 / 75 [UNK] [UNK] 110 / 108 [UNK] [UNK] [UNK] ( [UNK] )
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Нокиан 265 / 60 / 18 V 110 HAKKA BLUE 2 SUV
[UNK] 265 / 60 / 18 [UNK] 110 [UNK] [UNK] 2 [UN

Note that there's another way to easily perform quick inference with a trained model: the [pipeline API](https://huggingface.co/docs/transformers/main_classes/pipelines). The pipeline API abstracts away all the complexity for you (basically performing what we did above). Here, we'll use the [TokenClassificationPipeline](https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/pipelines#transformers.TokenClassificationPipeline) since that's the task we're doing, and we provide a model and tokenizer.

In [37]:
from transformers import pipeline

pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
pipe("Viatti Bosco S / T V-526 215 / 65 / 16 T 98")

[]

In [38]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# model_name = "nielsr/bert-finetuned-ner"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name)